In [ ]:
from keras.preprocessing import image
from tqdm import tqdm

In [ ]:
# 테스팅할 때 '카테고리'와 '소매기장'에 각각 소속하는 클래스를 구분해내기 위한 커스텀 함수
def find_best_indices_for_each_big_class(sorted_indices):
    sorted_indices_arr = sorted_indices.tolist()
    highest_index = sorted_indices_arr[-1]
    leftovers = sorted_indices_arr[:-1]

    best = [highest_index]
    category_class = range(0,7)
    sleeve_class = range(7,13)
    for num in reversed(leftovers):
				# 가장 높아서 먼저 뽑힌 애가 '카테고리' 관련 클래스였다면
        if highest_index in category_class:
            if num in sleeve_class:
                best.append(num)
                break
        elif highest_index in sleeve_class: # '소매기장' 관련 클래스였다면
            if num in category_class:
                best.append(num)
                break
    return sorted(best)

correct = 0.
total = 0.

for counter, data in tqdm(enumerate(test_loader), total=int(len(test_data)/test_loader.batch_size) ):
    image = data['image'].to(device)
    target = data['label'].to(device)
    # get all the index positions where value == 1 (정답 가져오기)
    target_indices = [i for i in range(len(target[0])) if target[0][i] == 1]
    # get the predictions by passing the image through the model (예측값 가져오기)
    outputs = model(image)
    outputs = torch.sigmoid(outputs)
    outputs = outputs.detach().cpu()
    sorted_indices = np.argsort(outputs[0])
    # best = sorted_indices[-3:] # 이거 대신 아래처럼
		# 최상위 '카테고리'와 '소매기장' 1개씩 list에 담아 리턴
    best = find_best_indices_for_each_big_class(sorted_indices)

		# ACCURACY 찍기
    total += target.size(0) # accumulate to dataset total size
    # calculate how many images were correctly classified
    correct_count = len(set(target_indices).intersection(best))
    correct += correct_count

print("correct", correct, "total", total)
accuracy = 100 * correct / (total*2)
print("Accuracy: {}%".format(accuracy))